In [ ]:
import migYOLO.pipeline.pipeline as mp
import migYOLO.utils.readYAML as ry
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
import os
import torch

# Overview

The migYOLO pipeline contains tools to downsample and process images with YOLO. This tutorial will breakdown this process step-by-step. After this, we can run `process_images.py` to process all of our images. The other notebook in this directory, called `Migdal_skim_visualization.ipynb`, shows an example of analyzing YOLO's outputs.

Let's start by loading `globalConf.yaml` to get a sense of the parameters you can adjust in this file

In [ ]:
conf = ry.read_config_file('globalConf.yaml')

In [ ]:
'''conf has two keys that can be read as conf['yoloConf'] and conf['downsample']
Open up globalConf.yaml in a text editor for more detailed descriptions of each key'''
conf

### GPU test: If you're planning to run with GPU support, the following cell should output True. If it does not, then your PyTorch GPU support isn't correctly configured. Troubleshooting this is an issue with PyTorch and not migYOLO

In [ ]:
torch.cuda.is_available()

### Now let's load some images

In [ ]:
"""Let's see the image files we have"""
os.listdir(conf['downsample']['data_dir'])

In [ ]:
"""We'll just load the first image"""
ims = io.imread(conf['downsample']['data_dir']+'/Images_batch_1.MTIFF',plugin='tifffile')

In [ ]:
'''We can see the dimensions of the images using np.shape(). The output tells us we have
200 images of dimension 2048 x 1152'''
np.shape(ims)

In [ ]:
'''This is a raw image without any processing'''

plt.imshow(ims[3])
plt.colorbar()

In [ ]:
'''We can use the mp.downsample class to process our image
It can read in a single image or a whole stack of them'''
ds = mp.downsample(ims[3])

In [ ]:
'''ds is an instance of the downsample object acting on ims[0]
The functions of note here are ds.downsampledImages and ds.processedImages'''
downsample = ds.downSampledImages
processed = ds.processedImages

In [ ]:
plt.figure(figsize = (18,9))

'''dark subtracted and 4x4 binned'''
plt.subplot(2,1,1)
plt.imshow(downsample,cmap = 'jet')
plt.xlabel('x')
plt.ylabel('y')
plt.title('4x4 binned')
plt.colorbar().set_label('Intensity [ADU]',rotation = 270,labelpad = 20)

'''dark subtracted, 4x4 binned, and Gaussian filtered'''
plt.subplot(2,1,2)
plt.imshow(processed,cmap = 'jet')
plt.xlabel('x')
plt.ylabel('y')
plt.title('4x4 binned with Gaussian blurring')
plt.colorbar().set_label('Intensity [ADU]',rotation = 270,labelpad = 20)

plt.tight_layout()
plt.show()

In [ ]:
'''We can do the same with the entire batch of images (takes a bit of time on a CPU, very quick on a modern GPU)'''
ds_all = mp.downsample(ims)

In [ ]:
'''You can see that we now have 200 processed images of shape 512 x 288'''
np.shape(ds_all.processedImages)

In [ ]:
'''Lets look at a random processed_image'''
processed_ims = ds_all.processedImages

plt.imshow(processed_ims[1],cmap = 'jet')
plt.xlabel('x')
plt.ylabel('y')
plt.title('4x4 binned with Gaussian blurring')
plt.colorbar().set_label('Intensity [ADU]',rotation = 270,labelpad = 20)

# Now let's process these images with YOLO

mp.yolo takes 4x4 Gaussian filtered images as input. conf['yoloConf'] has many of the important input parameters. **Remember, you can edit all of these parameters in `globalConf.yaml`**. If you're using the pretrained models
that come with migYOLO, do not edit `png_threshold` or `png_saturation`.

mp.yolo is a high-level function that processes each image and outputs pandas dataframes containing physical quantities of interest. If you would like more flexibility in what YOLO outputs, please consult the [Ultralytics YOLOv8 documentation](https://github.com/ultralytics/ultralytics)

In [ ]:
conf['yoloConf']

In [ ]:
np.expand_dims(processed_ims[51],axis=0)

In [ ]:
'''YOLO currently does not have support to be run on one image at a time. Two images at a time is okay, however. It
is most efficient to run on all 200 images (provided there is enough memory to handle it). YOLO takes a while 
to run on a cpu (up to a few minutes), while it is very fast on a modern GPU (a few seconds or less)'''

sample = processed_ims #If you wanted to run on a subset you could set sample = processed_ims[i:j]; with j-i > 1

yolo = mp.yolo(
        infile = sample, #This can either be a numpy array or a .npy/.npz file
        outpath = None, #If outpath is not specified, YOLO's output will not be saved
        outfilename = None, #Should be a name with .feather at the end if outpath is not None
        model = YOLO(conf['yoloConf']['model']), #Ultralytics' YOLO() wraps around our model which is augment.pt
        vignetting_map_file = conf['yoloConf']['vignetting_map'], #this is to correct toward vignetting away from the center of the readout
        calibration_file = conf['yoloConf']['calibration_file'], #this is to calibrate energy
        png_threshold = conf['yoloConf']['png_threshold'], #YOLO is trained on log scale images, this is the threshold intensity on a logarithmic scale
        png_saturation = conf['yoloConf']['png_saturation'], #threshold max intensity on a log scale. If you're using base.pt or augment.pt, don't change this or png_threshold
        remove_downsample = False, #If infile were a .npy or .npz filepath, setting this to True would delete that file
        save_pixels = conf['yoloConf']['save_pixels'], #True saves the pixel data within each bounding box. Recommend False to keep outfile sizes smaller
        migdal_cut = conf['yoloConf']['migdal_cut']) #Migdal search criteria

In [ ]:
'''The output of YOLO is a pandas dataframe'''
out = yolo.data

'''Migdal candidates are found in yolo.comb. yolo.comb will be empty if no candidates satisfying the
criteria entered into the migdal_cut argument of mp.yolo() are found'''

cand = yolo.comb

In [ ]:
"""Let's see the content of our output"""
print(out.columns,len(out))

In [ ]:
'''Lets see our output species'''
output_map = {0:'ER',1:'Hot_pix',2:'NR',3:'proton',4:'proton_ag',5:"shutter_clip",
             6:'spark',7:'spark_ag',8:'storm'}
fig,ax = plt.subplots()
ax.hist(out['prediction']-0.25,bins = 18, range = (-0.25,8.75))
ax.set_xticks([i for i in range(0,9)])
ax.set_xticklabels([output_map[i] for i in range(0,9)],rotation=75)
plt.show()

In [ ]:
'''NR ghosts (Section III of paper) are initially predicted to be ERs (prediction = 0) by YOLO, however we have
a flag for these afterglows that mp.yolo() computes. Let"s change the prediction of NR ghosts from 0 to 9'''

#Query events that are flagged as NR ghosts (AG stands for "afterglow")
out.query('AG_flag == 1')

#Change the prediction to 9
index = out.query('AG_flag == 1').index.to_numpy() #get index of afterglows
out['prediction'][index] = 9 #set afterglow prediction to 9

In [ ]:
'''Now lets plot with NR ghosts labeled as NR_ag. IMPORTANT: since image frames in our sample were
pulled at random from a much larger set of data, the parent frames of ghost events 
will not be present in these samples'''
output_map = {0:'ER',1:'Hot_pix',2:'NR',3:'proton',4:'proton_ag',5:"shutter_clip",
             6:'spark',7:'spark_ag',8:'storm',9:'NR_ag'}
fig,ax = plt.subplots()
ax.hist(out['prediction']-0.25,bins = 20, range = (-0.25,9.75))
ax.set_xticks([i for i in range(0,10)])
ax.set_xticklabels([output_map[i] for i in range(0,10)],rotation=75)
plt.show()

### Finally, let's plot some bounding boxes on the parent frames

In [ ]:
'''original_index tells us which frame the image belonged to

Recall: ims = raw images
        sample = processed images

'''    

'''im_array is the array of images
i is the frame index
yolo_scale if set to True, plots the downsampled image on the log scale that"s fed into YOLO
conf_thresh is the YOLO classification confidence threshold. min is 0, max is 1'''

def plot(im_array,i,yolo_scale,conf_thresh = 0):
    tmp = out.query('original_index == %s & prob >= %s'%(i,conf_thresh)) #grab the ith frame from YOLO's output dataframe
    if len(tmp) == 0:
        raise ValueError("No YOLO events for frame %s"%(i))
    max_dim = np.shape(im_array)[2]
    
    '''Multiply bounding box perimeters by 4 if raw image'''
    if max_dim == 2048:
        if yolo_scale:
            raise ValueError("yolo_scale needs to be False for unprocessed images")
        for col in ['colmin','colmax','rowmin','rowmax']:
            tmp[col] = tmp[col] * 4
    elif max_dim == 512:
        pass
    else:
        raise ValueError("This code was made for 2048 x 1152 or 512 x 288 images")
    
    plt.figure(figsize = (9,4.5))
    
    #Plot image
    if not yolo_scale:
        plt.imshow(im_array[i],cmap='jet')
    else:
        im = np.copy(im_array[i])
        im[im<0] = 0
        plt.imshow(np.log10(im+1),cmap='jet',
                   vmin=conf['yoloConf']['png_threshold'],
                  vmax=conf['yoloConf']['png_saturation'])
    
    color_map = {0:'pink',1:'cyan',2:'red',3:'yellow',4:'goldenrod',5:"white",
             6:'green',7:'forestgreen',8:'magenta',9:'maroon'}
    #Plot bounding boxes
    for cmin,cmax,rmin,rmax,pred in zip(tmp['colmin'],tmp['colmax'],
                                         tmp['rowmin'],tmp['rowmax'],tmp['prediction']):
        plt.hlines(rmin,cmin,cmax,color=color_map[pred],lw=2)
        plt.hlines(rmax,cmin,cmax,color=color_map[pred],lw=2)
        plt.vlines(cmin,rmin,rmax,color=color_map[pred],lw=2)
        plt.vlines(cmax,rmin,rmax,color=color_map[pred],lw=2)
        
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar().set_label('Intensity [ADU]',rotation = 270, labelpad = 20)
    plt.show()

In [ ]:
cand

In [ ]:
'''Raw image'''
plot(ims,1,yolo_scale=False,conf_thresh = 0)

In [ ]:
'''Processed image'''
plot(sample,1,yolo_scale=False,conf_thresh = 0.4)

In [ ]:
'''Processed image as YOLO sees it'''
plot(sample,1,yolo_scale=True,conf_thresh = 0.4)

### Now that you have some familiarity of how to downsample images and process them with YOLO, please run `process_images.py` and then you can move on to `Migdal_skim_visualization.ipynb`